**CS231A Final Project**

data

connect to drive

In [ ]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the .py files in 'p3/code' needed for this problem
# e.g. 'cs231a/monocular_depth_estimation':
FOLDERNAME = 'cs231a_final_pj'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%ls .
%cd drive/MyDrive
%cd $FOLDERNAME

Mounted at /content/drive
drive/  sample_data/
/content/drive/MyDrive
/content/drive/.shortcut-targets-by-id/1T0U6l3HfS0JC4GM27mZEFfQKRMYMPHTH/cs231a_final_pj


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1T0U6l3HfS0JC4GM27mZEFfQKRMYMPHTH/cs231a_final_pj


In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import itertools

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import cv2
import matplotlib.image
from PIL import Image

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only.")

transform = transforms.Compose([
    transforms.Resize((512, 1024)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111
Using the GPU!


In [ ]:
# extract the label training images
def load_data_labelID(path_label, subfolder, transform, batch_size, shuffle=False):
    # create the label dataset
    dataset = datasets.ImageFolder(path_label, transform)
    index = dataset.class_to_idx[subfolder]
    n = 0
    m = 0
    
    for i in range(dataset.__len__()):
        if index != dataset.imgs[n][1]:
            del dataset.imgs[n]
            n = n - 1
        else:
            if m % 3 != 2:
                
                del dataset.imgs[n]
                n = n - 1
            m = m + 1
        n = n + 1
    len_dataset = dataset.__len__()
    Dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, pin_memory=True)
    return (len_dataset, Dataloader)


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, pic):
        return torch.tensor(np.array(pic))


def load_data_labelID_test(path_label, subfolder, transform, batch_size, shuffle=False):
    # create the label dataset

    def pil_loader(path):
      # with open(path, 'rb') as f:
      img = Image.open(path)
      # print(np.array(img))
      return img

    my_transform = transforms.Compose([ToTensor()])
  
    dataset = datasets.ImageFolder(path_label, transform=my_transform, loader=pil_loader)
    index = dataset.class_to_idx[subfolder]
    n = 0
    m = 0
    
    for i in range(dataset.__len__()):
        if index != dataset.imgs[n][1]:
            del dataset.imgs[n]
            n = n - 1
        else:
            if m % 3 != 2:
                
                del dataset.imgs[n]
                n = n - 1
            m = m + 1
        n = n + 1
    len_dataset = dataset.__len__()
    Dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, pin_memory=True)
    return (len_dataset, Dataloader)


# extract the label training images
def load_data_label(path_label, subfolder, transform, batch_size, shuffle=False):
    # create the label dataset
    dataset = datasets.ImageFolder(path_label, transform)
    index = dataset.class_to_idx[subfolder]
    n = 0
    m = 0
    for i in range(dataset.__len__()):
        if index != dataset.imgs[n][1]:
            del dataset.imgs[n]
            n = n - 1
        else:
            if m % 3 != 0:
                del dataset.imgs[n]
                n = n - 1
            m = m + 1
        n = n + 1
    len_dataset = dataset.__len__()
    Dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, pin_memory=True)
    return (len_dataset, Dataloader)


# extract raw training images
def load_data_raw(path, subfolder, transform, batch_size, shuffle=False):
    dataset = datasets.ImageFolder(path, transform)
    index = dataset.class_to_idx[subfolder]
    n = 0
    for i in range(dataset.__len__()):
        if index != dataset.imgs[n][1]:
            del dataset.imgs[n]
            n = n - 1
        n = n + 1
    len_dataset = dataset.__len__()
    Dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, pin_memory=True)
    return (len_dataset, Dataloader)

In [ ]:
batch_size_train = 1

(len_train_label, train_label_loader) = load_data_labelID(
    'data/gtFine', 
    'train', transform, batch_size_train, shuffle=False
)
(len_train_raw, train_raw_loader) = load_data_raw(
    'data/leftImg8bit',
    'train', transform, batch_size_train, shuffle=False
)

model

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)

# define U-net
class Unet(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Unet, self).__init__()

        self.conv1 = DoubleConv(in_ch, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = DoubleConv(512, 1024)
        self.up6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv6 = DoubleConv(1024, 512)
        self.up7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv7 = DoubleConv(512, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv8 = DoubleConv(256, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv9 = DoubleConv(128, 64)
        self.conv10 = nn.Conv2d(64, 34, 1)

    def forward(self, x):
        c1=self.conv1(x)
        p1=self.pool1(c1)
        c2=self.conv2(p1)
        p2=self.pool2(c2)
        c3=self.conv3(p2)
        p3=self.pool3(c3)
        c4=self.conv4(p3)
        p4=self.pool4(c4)
        c5=self.conv5(p4)
        up_6= self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6=self.conv6(merge6)
        up_7=self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7=self.conv7(merge7)
        up_8=self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8=self.conv8(merge8)
        up_9=self.up9(c8)
        merge9=torch.cat([up_9, c1], dim=1)
        c9=self.conv9(merge9)
        c10=self.conv10(c9)
        out = c10  #nn.Sigmoid()(c10)
        return out

train

In [ ]:

# transform testing datasets' H reduced img(0-420) to (512, 1024)
def crop_tensor_test(x, y):
    cropped_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((512, 1024)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])

    batch_size = y.shape[0]
    x_temp, y_temp = x[:, :, :420, :], y[:, :, :420, :]
    x0, y0 = cropped_transform(x_temp[0]), cropped_transform(y_temp[0])
    x1, y1 = cropped_transform(x_temp[1]), cropped_transform(y_temp[1])
    x2, y2 = cropped_transform(x_temp[2]), cropped_transform(y_temp[2])
    x3, y3 = cropped_transform(x_temp[3]), cropped_transform(y_temp[3])
    x4, y4 = cropped_transform(x_temp[4]), cropped_transform(y_temp[4])
    x_ = torch.cat((x0.unsqueeze(0), x1.unsqueeze(0), x2.unsqueeze(0), x3.unsqueeze(0), x4.unsqueeze(0)), 0)
    y_ = torch.cat((y0.unsqueeze(0), y1.unsqueeze(0), y2.unsqueeze(0), y3.unsqueeze(0), y4.unsqueeze(0)), 0)  #(5, 3, 512, 1024)

    x_, y_ = x_.cuda(), y_.cuda()


    return (x_, y_)



# show images
def process_image(img):
    img = img.cpu().data.numpy().transpose(1, 2, 0)
    return img


# save the training results
def save_visualized_result(model, x, y, num_epoch):
    (x_, y_) = crop_tensor_test(x, y)
    predict_label = model(x_)
    (temp_max, preds) = torch.max(predict_label.data, 1)  #preds->(5, 512, 1024)
    #print(preds.shape)

    fig, ax = plt.subplots(x_.size()[0], 3, figsize=(12, 10)) # figsize ratio (w,h) (18, 15)--->(5)

    for i in range(x_.size()[0]):
        ax[i, 0].get_xaxis().set_visible(False)
        ax[i, 0].get_yaxis().set_visible(False)
        ax[i, 1].get_xaxis().set_visible(False)
        ax[i, 1].get_yaxis().set_visible(False)
        ax[i, 2].get_xaxis().set_visible(False)
        ax[i, 2].get_yaxis().set_visible(False)
        ax[i, 0].cla()
        ax[i, 0].imshow(process_image(x_[i]))
        ax[i, 1].cla()
        ax[i, 1].imshow(visualize_prediction(preds[i]))
        ax[i, 2].cla()
        ax[i, 2].imshow(process_image(y_[i]))

    plt.tight_layout()
    label_epoch = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0, label_epoch, ha='center')
    label_input = 'Input'
    fig.text(0.18, 1, label_input, ha='center')
    label_output = 'Output'
    fig.text(0.5, 1, label_output, ha='center')
    label_truth = 'Truth'
    fig.text(0.81, 1, label_truth, ha='center')

    plt.savefig(
        'Training_results/Epoch %d.png' %
        (num_epoch), bbox_inches='tight'
    )#, bbox_inches='tight')

    plt.close()



# visualize the results of categories  (512, 1024)->(512, 1024, 3)
def visualize_prediction(output_model):
    predict_img = output_model.cpu().data.numpy()
    #print(predict_img)

    color_array = np.array([
        [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [111, 74, 0],
        [81, 0, 81], [128, 64, 128], [244, 35, 232], [250, 170, 160], [230, 150, 140], [70, 70, 70],
        [102, 102, 156], [190, 153, 153], [180, 165, 180], [150, 100, 100], [150, 120, 90], [153, 153, 153],
        [153, 153, 153], [250, 170, 30], [220, 220, 0], [107, 142, 35], [152, 251, 152], [70, 130, 180],
        [220, 20, 60], [255, 0, 0], [0, 0, 142], [0, 0, 70], [0, 60, 100], [0, 0, 90],
        [0, 0, 110], [0, 80, 100], [0, 0, 230], [119, 11, 32]
    ], dtype=np.uint8)

    visualization = np.zeros((512, 1024, 3), dtype=np.uint8)
    for h in range(predict_img.shape[0]):
        for w in range(predict_img.shape[1]):
            visualization[h][w] = color_array[(int)(predict_img[h][w])]
            #visualization[h][w][1] = color_array[predict_img[h][w]][1]
            #visualization[h][w][2] = color_array[predict_img[h][w]][2]

    return visualization


In [ ]:
# show the training result
def show_result(model, x_, y_, num_epoch):
    predict_images = model(x_)

    fig, ax = plt.subplots(x_.size()[0], 3, figsize=(12, 10))

    for i in range(x_.size()[0]):
        ax[i, 0].get_xaxis().set_visible(False)
        ax[i, 0].get_yaxis().set_visible(False)
        ax[i, 1].get_xaxis().set_visible(False)
        ax[i, 1].get_yaxis().set_visible(False)
        ax[i, 2].get_xaxis().set_visible(False)
        ax[i, 2].get_yaxis().set_visible(False)
        ax[i, 0].cla()
        ax[i, 0].imshow(process_image(x_[i]))
        ax[i, 1].cla()
        ax[i, 1].imshow(process_image(predict_images[i]))
        ax[i, 2].cla()
        ax[i, 2].imshow(process_image(y_[i]))

    plt.tight_layout()
    label_epoch = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0, label_epoch, ha='center')
    label_input = 'Input'
    fig.text(0.18, 1, label_input, ha='center')
    label_output = 'Output'
    fig.text(0.5, 1, label_output, ha='center')
    label_truth = 'Truth'
    fig.text(0.81, 1, label_truth, ha='center')

    plt.show()

# count parameters
def count_params(model):
    num_params = sum([item.numel() for item in model.parameters() if item.requires_grad])
    return num_params

# transform training datasets' H reduced img(0-420) to (512, 1024)
def crop_tensor_train(x, y):
    cropped_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((512, 1024)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])

    batch_size = y.shape[0]
    x_temp, y_temp = x[:, :, :420, :], y[:, :, :420, :]
    x0, y0 = cropped_transform(x_temp[0]), cropped_transform(y_temp[0])
    x0 = x0.unsqueeze(0)

    # x_, y_ = x0.cuda(), y0.cuda()

    x_, y_ = x0.to(device), y0.to(device)
    return (x_, y_)

# train process
def train_model(model, criterion, optimizer, train_label_loader, train_raw_loader, test_raw, test_label, current_epoch=0, num_epochs=20):
    start_time = time.time()
    # loss history
    hist_losses = []

    for epoch in range(current_epoch, num_epochs):
        print('Start training epoch %d' % (epoch + 1))
        losses_list = []
        epoch_start_time = time.time()
        num_iter = 0
        for (train_label, train_raw) in zip(train_label_loader, train_raw_loader):
            y, temp1 = train_label  #torch.Size([1, 3, 512, 1024])
            x, temp2 = train_raw  #torch.Size([1, 3, 512, 1024])
            y = y.to(device)
            temp1 = temp1.to(device)
            x = x.to(device)
            temp2 = temp2.to(device)

            #print(y.shape)
            #print(torch.sum(torch.sum(y, axis=2), axis=2))

            x_, y_crop = crop_tensor_train(x, y)  #y_->(3, 512, 1024)
            #print(y_.shape)
            y_temp = y_crop[0].unsqueeze(0)
            #print(y_temp.shape)
            y_ = (y_temp * 255).long()
            #print(x_.shape)
            #print(y_.shape)
            #print(torch.max(y_))

            # Compute loss
            # forward
            outputs = model(x_)
            #print(preds)
            #print(preds.shape)
            loss = criterion(outputs, y_)
            # Bp and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            losses_list.append(loss)
            hist_losses.append(loss)
            num_iter += 1

        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print('[%d/%d] - using time: %.2f' % ((epoch + 1), num_epochs, per_epoch_ptime))
        print('Loss: %.3f' % (torch.mean(torch.FloatTensor(losses_list))))
        
        model.eval()
        with torch.no_grad():
            save_visualized_result(model, test_raw, test_label, (epoch + 1))

        # save train result
        torch.save(model.state_dict(), 'weights/weights_epoch%d.pth' % (epoch+1))
        print("Weights saved!")

    end_time = time.time()
    total_ptime = end_time - start_time


    return (model, hist_losses)


# train the network
def train(model, train_label_loader, train_raw_loader, test_raw, test_label,current_epoch=0, num_epochs=20):
    # define LOSS functions
    criterion = nn.CrossEntropyLoss()  #nn.BCELoss().cuda()
    # Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.5, 0.999))  #lr = 1e-5  fair:0.0002 & (0.5, 0.999)
    print('Training start!')
    (model, hist_losses) = train_model(
        model, criterion, optimizer,
        train_label_loader, train_raw_loader, test_raw, test_label, current_epoch, num_epochs
    )


    return (model, hist_losses)


In [ ]:
model = Unet(3, 34)
print(device)
model.load_state_dict(torch.load('./weights/weights_epoch1001.pth'))
model = model.to(device)
model.train()
# report the architectures of Unet
# print(model)
print('Number of trainable parameters {}'.format(count_params(model)))
# model.load_state_dict(torch.load('weights/weights_epoch10.pth'))
"""
wandb.init(
      # Set the project where this run will be logged
      project="U-Net", 
      # Track hyperparameters and run metadata
      config={
      "architecture": "UNet",
      "dataset": "Cityscapes",
      "epochs": 105,
})
"""
# train
(model, hist_losses) = train(
    model, train_label_loader, train_raw_loader, test_raw, test_label,current_epoch = 1000, num_epochs=1005
)


In [ ]:
model = Unet(3, 34)
print(device)
model = model.to(device)
model.load_state_dict(torch.load('./weights/weights_epoch1001.pth'))
model.eval()
with torch.no_grad():
  save_visualized_result(model, test_raw, test_label, 10)

cuda:0


test

In [ ]:
def mIOU(target, prediction, num_class):
    #Input: target & prediction: numpy H*
    #Output: iou_scores: each classes' iou,ave: mIoU
    iou_score = []
    iou_scores = np.zeros(num_class,)
    for i in range(num_class):
        t = (target==i)
        p = (prediction==i)
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        if np.sum(union) != 0:
            iou_score.append(np.sum(intersection) / np.sum(union))
            #iou_scores[i] = np.sum(intersection) / np.sum(union)
        iou_scores[i] = np.sum(intersection) / np.sum(union)
    ave = np.mean(iou_score)

    return (iou_scores, ave)

# resize image for compute accuracy (512, 1024, 3)->(1024, 2048, 3)
def pyr_up_IOU(p):
    p_ = p.copy()
    return cv2.resize(p_, (2048, 1024), interpolation=cv2.INTER_NEAREST)

In [ ]:
model = Unet(3, 34)
model.load_state_dict(torch.load('./weights/weights_epoch1004.pth'))
model = model.cuda()

for i in range(0, 100):
    n = str(i)
    s = n.zfill(3)
    print(s)
    img_raw_path = './image_2/000'+s+'.png'

    img_raw = Image.open(img_raw_path)
    img_input = transform(img_raw).unsqueeze(0).to(device)
    predict_label = model(img_input)

    (temp_max, preds) = torch.max(predict_label.data, 1)  
    preds_original = preds[0]
    preds_original[preds_original <= 6] = 0
    preds_original[(preds_original >= 7) & (preds_original <= 10)] = 7
    preds_original[(preds_original >= 11) & (preds_original <= 16)] = 11
    preds_original[(preds_original >= 17) & (preds_original <= 20)] = 17
    preds_original[(preds_original >= 21) & (preds_original <= 22)] = 21
    preds_original[(preds_original >= 24) & (preds_original <= 25)] = 24
    preds_original[preds_original > 26] = 26

    temp = Image.fromarray(visualize_prediction(preds_original)).resize((1242,375))

    temp.save('./merge_kitti_rgb/000'+s+'.png')

In [ ]:
(len_test_label, test_label_loader) = load_data_labelID_test(
    'data/gtFine',
    'val', None, 1, shuffle=False
)


(len_test_raw, test_raw_loader) = load_data_raw(
    'data/leftImg8bit',
    'val', transform, 1, shuffle=False
)

In [ ]:
imgPredicts = []
imgLabels = []
imgIoU = []
model = Unet(3, 34)
iou_scores_all = np.zeros(34,)

model.load_state_dict(torch.load('./weights/weights_epoch1001.pth'))
model = model.cuda()
n = 0
with torch.no_grad():
  for (test_label, test_raw) in zip(test_label_loader, test_raw_loader):
    
    # print(test_label[0].shape) #torch.Size([5, 3, 512, 1024])
    y, _ = test_label  
    x, _ = test_raw  
    # y = y.to(device)
    x = x.to(device)

    # print(x.shape)
    # print(x)
    predict_label = model(x)
    (temp_max, preds) = torch.max(predict_label.data, 1)
    imgPredict = pyr_up_IOU(preds[0].cpu().data.numpy())
    imgLabel = np.array(y)
    (iou_scores, ave) = mIOU(imgLabel, imgPredict, 34)
    iou_scores[np.isnan(iou_scores)] = 0
    print(iou_scores)
    print(ave)
    iou_scores_all += iou_scores
    n += 1
    # break

  # (iou_scores, ave) = mIOU(imgLabel, imgPredict, 34)
  # imgIoU.append(iou_scores)
  
  
  # imgLabels.append(imgLabel)




In [ ]:
from collections import namedtuple

Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

In [ ]:
class_label_map = {}
iou_scores_all = list(iou_scores_all/n)
for lable in labels:
  class_label_map[lable.name] = iou_scores_all[lable.id]

In [ ]:
sorted(class_label_map.items(), key=lambda item: item[1])

[('unlabeled', 0.0),
 ('tunnel', 0.0),
 ('polegroup', 0.00018896655891022308),
 ('rail track', 0.00029594764278822143),
 ('caravan', 0.0005389636375379433),
 ('trailer', 0.0006558250949273784),
 ('guard rail', 0.0007204225352112676),
 ('bridge', 0.002286358954961396),
 ('ego vehicle', 0.003591172527261949),
 ('train', 0.009844163072259915),
 ('truck', 0.02032375993474554),
 ('ground', 0.024548623142727353),
 ('motorcycle', 0.03189101395059155),
 ('dynamic', 0.03388884785619496),
 ('parking', 0.04319210060649177),
 ('bus', 0.0581834525965247),
 ('wall', 0.06947129359680092),
 ('fence', 0.07859418618848754),
 ('rectification border', 0.08779596365227266),
 ('rider', 0.10898917694043372),
 ('terrain', 0.13617073331951968),
 ('static', 0.13670155540171416),
 ('traffic light', 0.14964619726474673),
 ('bicycle', 0.24959546028473564),
 ('license plate', 0.24959546028473564),
 ('person', 0.3373705595352572),
 ('pole', 0.37438479739827385),
 ('traffic sign', 0.4217815570912375),
 ('sidewalk', 0